## This script finds the mean distance from a resident of an informal or atomistic area to the nearest open space.

### Method:

1. Reclass ULU: [2, 3] --> [1, 1], and 0 otherwise. This gives 1 for informal/atomistic. Call this **informal**.
1. Multiply WorldPop raster by **informal**. This gives popsize only in informal pixels, zero otherwise. Call this **informal_pop**.
1. Create distance raster giving distance within 400m of OSM openspace features. Call this **openspace_distance**.
1. Multiply **informal_pop** by **openspace_distance**. Call this **informal_persondistancetoopenspace**.
1. Sum over **informal_pop** to get total number of residents of informal/atomistic. Call this **total_informal_pop**.
1. Return **informal_persondistancetoopenspace** / **total_informal_pop**. This is the average distance from an informal/atomistic dwelling to openspace.


### To do:
* Return *nodata* if city not covered by ULU dataset

In [4]:
import os, requests, json
import pandas as pd
import geemap
import ee
ee.Authenticate()

In [5]:
ee.Initialize()

In [6]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-indicators'
aws_s3_dir = "https://"+bucket_name+".s3.eu-west-3.amazonaws.com"

In [7]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM3,COD,ADM3
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2


In [15]:
POP_YEAR = 2020
OPENSPACE_YEAR = 2022

In [9]:
# Get ULU polygons

ULU200 = ee.ImageCollection('projects/wri-datalab/urban_land_use/V1')
ULU4000 = ee.ImageCollection('projects/wri-datalab/cities/urban_land_use/V1')
ULUv2 = ee.ImageCollection('projects/wri-datalab/urban_land_use/V2')
ULU = ULU200.merge(ULU4000).merge(ULUv2)
ULU = ULU.select('lulc').reduce(ee.Reducer.firstNonNull()).rename('lulc') #//.clip(cityArea)//.updateMask(ULUMexico)
informal = ULU.mask(ULU.mask().gt(0)).remap([2, 3], [1, 1], 0)

In [10]:
# Get WorldPop data

pop = ee.ImageCollection('WorldPop/GP/100m/pop').filter(ee.Filter.equals('year', POP_YEAR))

In [14]:
def do_one_row(i):
    
    boundary_id_aoi = '{0}-{1}'.format(boundary_georef.loc[i, 'geo_name'], boundary_georef.loc[i, 'aoi_boundary_name'])
    boundary_path = '{0}/data/boundaries/boundary-{1}.geojson'.format(aws_s3_dir, boundary_id_aoi)
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    country_code = boundary_georef.loc[i, 'country_code']

    # Get open space polygons
    openspace_path = aws_s3_dir + '/data/open_space/openstreetmap/{0}-OSM-open_space-{1}.geojson'.format(boundary_id_aoi, OPENSPACE_YEAR)
    openspace_geo = requests.get(openspace_path).json()
    openspace_geo_ee = geemap.geojson_to_ee(openspace_geo)
    openspace_distance = openspace_geo_ee.distance(20000, 10)
    
    localpop = pop.filter(ee.Filter.equals('country', country_code)).select('population').first().clip(boundary_geo_ee)
    informal_pop = localpop.multiply(informal)
    
    informal_persondistancetoopenspace = (informal_pop.multiply(openspace_distance)).reduceRegion(reducer= ee.Reducer.sum(), geometry= boundary_geo_ee.geometry()).getInfo()['population']
    total_informal_pop = informal_pop.reduceRegion(reducer= ee.Reducer.sum(), geometry= boundary_geo_ee.geometry()).getInfo()['population']
    
    if total_informal_pop > 0:
        return informal_persondistancetoopenspace / total_informal_pop
    else:
        return -9999

In [16]:
do_one_row(10)

381.0627717218497